# Ejemplo(Falta agregar un costo de almacenamiento)
Juan vende cervezas artesanales de Lunes a Viernes. Las compra por 300 pesos y vende por 1.500 pesos cada una
Juan maneja su inventario de forma semanal, de la siguiente manera:
* Juan parte la semana con una cantidad de cervezas en inventario, y vende lascervezas durante la semana.
    
* Si un cliente quiere comprar una cerveza pero no hay disponible, Juan consideraque pierde 300 pesos por cada cerveza no vendida por falta de clientes.
    
* Juan vende entre 10 y 40 cervezas por semana.
    
* El Viernes al final del día Juan hace la contabilidad de costos, y cuenta cuantas unidades tiene en inventario. Según su nivel de inventario, tiene que tomar una decisión.
    
    * Compra lo suficiente para llenar su frigobar, que tiene una capacidad de S unidades.
    * No hacer ningún pedido esta semana.
* El pedido llega el Lunes en la mañana.
* S=50

## ¿Ayude a decidir a Juan cuando comprar, según el inventario que tenga?

In [1]:
import numpy as np
from numpy import random
import collections

In [2]:
# Información problema
S=500#20#0 #capacidad max y tamaño de lote
a=100#0#3#0 #Demanda minima
b=400#15#0 #Demanda maxima
cost=300#precio cerveza y costo perdida clientes
price=1500


#### Generó las demandas

In [5]:
random.seed(10)
total=50000
x=random.randint(a,b+1, size=(total))
Dem_frec = collections.Counter(x)
Dem_frec

Counter({365: 138,
         225: 152,
         115: 169,
         223: 177,
         256: 181,
         321: 170,
         108: 175,
         173: 171,
         356: 160,
         140: 149,
         116: 141,
         339: 188,
         154: 193,
         222: 181,
         162: 185,
         133: 145,
         300: 182,
         277: 161,
         279: 170,
         177: 154,
         113: 175,
         343: 162,
         241: 175,
         386: 164,
         258: 174,
         189: 171,
         240: 184,
         131: 171,
         392: 177,
         383: 178,
         118: 155,
         193: 163,
         194: 146,
         348: 168,
         174: 176,
         316: 171,
         218: 174,
         171: 157,
         245: 145,
         235: 193,
         303: 162,
         384: 169,
         389: 175,
         350: 153,
         144: 159,
         233: 182,
         104: 182,
         282: 167,
         390: 150,
         371: 171,
         243: 146,
         234: 152,
         346

### Comiezo a generar mi matriz de probaibilidades
Esta matriz se estructura de la siguiente forma

    Prob[i][j]: es la probailidad de que el inventario i pase al nivel de inventario j en el siguente periodo


In [6]:
prob =np.zeros([S+1,S+1])

In [6]:
prob#[5][5]=1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [7]:
for i in range(S+1):
    for j in range(S+1):
        dem=i-j
        if(S>=dem>=0):
           prob[i][j] =Dem_frec[dem]/total
    

In [46]:
prob

array([[0.095, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.08 , 0.095, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.055, 0.08 , 0.095, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.045, 0.055, 0.08 , 0.095, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.045, 0.045, 0.055, 0.08 , 0.095, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   ],
       [0.045, 0.045, 0.045, 0.055, 0.08 , 0.095, 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   

### primera idea
Esta matriz de probabilidades no sirve para calcular el rebenue esperado, porque no representa la demanda cuando sobrepasa el nivel de stock

Para almacenar el expected reward, se utilizará al siguiente estructura:

    reward[i][d]:= el expected reward de tomar la decisión d ante un nivel de inventario i.
    
tomando en cuenta que d, puede tener valores de 0 o 1., sea 0 no realizar la compra y 1 realizar la compra de inventario.

## Esta idea es mejor
tomando en cuenta que realizar la compra en el periodo t, donde el nivel de inventarios es de i, no afectaria al expected reward del mismo periodo t, ya que la compra afecta al nivel de inventarios del perido siguente. Es por esto que la desición no afecta al expected reward del mismo periodo en donde se toma.

Es por ello que para almacenar el expected reward, se utilizará al siguiente estructura:
    
    reward[i]:= el expected reward ante un nivel de inventario i.


In [18]:
# aquí se ajustan los costos de
# D>i o D<i

reward=np.zeros(S+1)
for i in range(S+1):
    r=0
    for dem in range(a,b+1):
        p=Dem_frec[dem]/total
        r_dem=price*min(dem,i)
        if(dem>=i):
            r_dem+=-cost*(dem-i)
        else:#Agrego costo de inventario sobrante
            r_dem+= -(cost)*(i-dem)
        r+= p*(r_dem)
    #print(r)
    reward[i]=r

# Comenzamos a estructurar esto

## Value Determination Equations

In [9]:
#funcion para obtener el valor de la ecuaciones
def Value_Determination_Equations(decisions,prob,descuento,costo,revenues):
    L=len(decisions)
    aux=np.zeros([L,L])
    B=np.zeros(L)
    for i in range(L):
        if decisions[i]:
        #Juan decide comprar mas cerveza.
            aux[i]=prob[L-1]
            B[i]=revenues[i]
            if(S-i>0):
            # si el nivel de inventario es menor al S el costo de pedir puede ser
                B[i]+=-costo*(S-i)
        else:
            aux[i]=prob[i]
            B[i]=revenues[i]
    A = np.identity(L)-(descuento*aux)
    X = np.linalg.solve(A,B)
    return X
    

In [53]:
D=np.zeros(501)
#D=np.array([0,0,1,1,1,1])

Value_Determination_Equations(D,prob,1,cost,reward)

array([-2481.21546961,  -900.54943378,   729.26982814,  2377.52118371,
        4019.48299955,  5653.54699081,  7196.73484534,  8853.36420813,
       10389.34892196, 11888.55091349, 13432.05358817, 14910.07957005,
       16375.57337829, 17784.0556333 , 19181.94325041, 20597.52185088,
       22146.8418072 , 23690.55691278, 25232.47103617, 26762.1817742 ,
       28282.56320117])

# Howard’s Policy Iteration Method

In [10]:
def T_delta(prob,values,descuento,costo,revenues):
    L=len(values)
    T=np.zeros(L)
    #aux_1 siempre igual
    aux_1=costo+(descuento*sum(prob[0]*values))
    for i in range(L):
        aux_2=revenues[i]+(descuento*sum(prob[i]*values))
        T[i]=max(aux_1,aux_2)
    return T
        
        

In [72]:
D=np.array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0., 0., 0., 0.])
D=np.array([1 for i in range(21)])
V_D_E=Value_Determination_Equations(D,prob,1,cost,reward)
V_D_E

array([-8.12125228e+19, -8.12125228e+19, -8.12125228e+19, -8.12125228e+19,
       -8.12125228e+19, -8.12125228e+19, -8.12125228e+19, -8.12125228e+19,
       -8.12125228e+19, -8.12125228e+19, -8.12125228e+19, -8.12125228e+19,
       -8.12125228e+19, -8.12125228e+19, -8.12125228e+19, -8.12125228e+19,
       -8.12125228e+19, -8.12125228e+19, -8.12125228e+19, -8.12125228e+19,
       -8.12125228e+19])

In [70]:
Los_T

array([-7.71518967e+18, -7.71518967e+18, -7.71518967e+18, -7.71518967e+18,
       -7.71518967e+18, -7.71518967e+18, -7.71518967e+18, -7.71518967e+18,
       -7.71518967e+18, -7.71518967e+18, -7.71518967e+18, -7.71518967e+18,
       -7.71518967e+18, -7.71518967e+18, -7.71518967e+18, -7.71518967e+18,
       -7.71518967e+18, -7.71518967e+18, -7.71518967e+18, -7.71518967e+18,
       -7.71518967e+18])

In [69]:
Los_T=T_delta(prob,V_D_E,1,cost,reward)
np.around(Los_T,4)<=np.around(V_D_E,4)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [11]:
def Search_OP(decisions,prob,descuento,costo,revenues):
   
    #(D,m_prob,descuento,c,states_reveneus)
    V_D_E = Value_Determination_Equations(decisions,prob,descuento,costo,revenues)
    Los_T=T_delta(prob,V_D_E,descuento,costo,revenues)
    val_Op=np.around(Los_T,4)<=np.around(V_D_E,4)
    if(sum(val_Op)==len(val_Op)):
        return(decisions)
    else:
        for i in range(len(val_Op)):
            if (val_Op[i]!=True):
                if(decisions[i]==1):
                    decisions[i]=0
                else:
                    decisions[i]=1
        #For 
        #print('recurción')
        return(Search_OP(decisions,prob,descuento,costo,revenues))

In [19]:
#D=np.array([1,0,0,0,0,1])
D=np.zeros(501)
D=np.array([1 for i in range(501)])
Optimo=Search_OP(D,prob,1,cost,reward)
Optimo

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [17]:
sum(Optimo)

501

In [14]:
D=np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1.])

In [15]:
#D=np.array([1,0,0,0,0,1])
#D=np.zeros(501)
#D=np.array([1 for i in range(501)])
Optimo=Search_OP(D,prob,1,cost,reward)
print(sum(Optimo))
Optimo

83.0


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.